# **Presentación del caso**

Predecir o estimar el precio de una vivienda puede ser de gran ayuda a la hora  de tomar decisiones importantes tales como la adquisición de casa propia . A continuación se presenta un dataset compuesto por **25, 660 registros** para **Argentina y Colombia** adjunto a las siguientes **10 variables**:

    1.   pais : "Argentina", "Colombia"
    2.   provincia_departamento: Provincia o departamento (no ambas)  donde se ubica el departamento
    3.   ciudad: Ciudad donde se ubica el departamento
    4.   property_type: "Departamento" (siempre en Argentina), "Apartamento" (siempre en Colombia)
    5.   operation_type: "Venta"
    6.   rooms: cantidad de espacios en general dentro del apartamento
    7.   bedrooms: cantidad de cuartos donde dormir dentro del apartamento
    8.   bathrooms: cantidad de baños dentro del apartamento
    9.   surface_total: área total en metros cuadrados del departamento
    10.  currency: "USD" (dólar americano)

![Image of Yaktocat](https://www.datasource.ai/uploads/7c2c64c37b855715637538ef4f19a46d.png)

# **Lectura de los datos**

In [1]:
"CELDA N°1"
#Importamos los datos de train y test considerando separador, índice y tipos de datos
import pandas as pd
data=pd.read_csv("https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/precios_argentina_colombia.csv",sep=";", index_col='Id')

In [2]:
"CELDA N°2"
#Comprobamos que la train tenga 25,660 filas y tanto train como test tengan 10 variables predictoras
data.shape

(25660, 11)

# **Eliminando variables no relevantes**

In [3]:
"CELDA N°3"
#Comprobamos que algunas columnas tienen un único valor en train
for col in data.columns:
  if data[col].nunique()<3:
    print('En la columna',col,'hay',data[col].nunique(),'valores distintos')
  else:
     pass #no realizar ninguna acción si es que el número de valores distintos es mayor que 2

En la columna pais hay 2 valores distintos
En la columna property_type hay 2 valores distintos
En la columna operation_type hay 1 valores distintos
En la columna currency hay 1 valores distintos


Las columnas **pais, property_type, operation_type, currency** tienen apenas 1 o 2 valores repetidos en toda la columna.

Por eso eliminamos estas columnas que no aportan data significativa para predecir el precio del departamento, **excepto la columna pais** porque será necesaria después para traer data externa.

In [4]:
"CELDA N°4"
#Eliminamos las columnas mencionadas con el método drop. No olvidar incluir inplace = True
data.drop(columns=['property_type', 'operation_type', 'currency'], inplace=True)

# **Preprocesamiento de los datos**

## I. Verificación de datos perdidos

In [5]:
"CELDA N°5"
#Verificamos que ninguna columna de train tenga vacíos
for col in data.columns:
  if data[col].isna().sum()>0:
    print('En la columna',col,'hay',data[col].isna().sum(),'valores nulos')

Si al ejecutar la celda anterior **no obtenemos resultado** se debe a que no se encontró ninguna columna que tenga datos perdidos.

## II. Verificación de outliers

Con ayuda del método **skew** descrita por la distribución de cada variable numérica detectaremos la presencia de **outliers**.

In [6]:
"CELDA N°6"
#Previamente seleccionamos las variables numéricas y en cada una calculamos el skew para mostrarlo en un dataframe
data_to_skew = data.select_dtypes(include = ["number"])

skew = []
for col in data_to_skew.columns:
  skew.append(data_to_skew[col].skew())

skewness=pd.DataFrame(index=data_to_skew.columns) #declaramos como índices a las columnas numéricas
skewness["Skewness"] = skew
skewness.sort_values(by=["Skewness"], ascending=False) #ordenamos de mayor a menor

,Skewness
price,6.026587
surface_total,2.081696
bathrooms,1.378065
rooms,1.222084
bedrooms,0.522811


Para corregir el **alto skewness** de las columnas **price y surface_total** aplicaremos una **transformación logarítmica**.

In [7]:
"CELDA N°7"
#Usamos una función lambda para realizar una transformación logarítmica usando numpy sobre todas las filas de la columna price
import numpy as np
data["price"] = data["price"].map(lambda i: np.log(i) if i > 0 else 0) 
print('El skewness después de la transformación logarítmica es: ',data["price"].skew())

El skewness después de la transformación logarítmica es:  0.8850647322573656


In [8]:
"CELDA N°8"
#Usamos una función lambda para realizar una transformación logarítmica usando numpy sobre todas las filas de la columna surface_total
data["surface_total"] = data["surface_total"].map(lambda i: np.log(i) if i > 0 else 0) 
print('El skewness después de la transformación logarítmica es: ',data["surface_total"].kurt())

El skewness después de la transformación logarítmica es:  -0.0539266496430324


## III. Feature Engineering

### *Añadimos data externa*

Juntando **país y provincia** obtenemos datos estadísticos sobre cada **provincia**

In [9]:
"CELDA N°9"
#Leemos la tabla externa pais_provincia con promedio, medianas y percentiles según provincia de Argentina y Colombia
pais_provincia=pd.read_csv("https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/pais_provincia.csv",sep=";")
pais_provincia.columns

Index(['pais_provincia', 'promedio_provincia', 'mediana_provincia',
       'percentil10_provincia', 'percentil25_provincia',
       'percentil75_provincia', 'percentil90_provincia'],
      dtype='object')

Añadimos **nuevas columnas** al final con ayuda del método **merge**. Previamente **concatenamos el país y provincia**

In [10]:
"CELDA N°10"
#Para concatenar ambas columnas simplemente usamos el operador + con el guión bajo entre comillas
data['pais_provincia'] = data['pais']+ "_" + data['provincia_departamento']

#Para adjuntar los datos de la tabla externa usamos el método merge especificando left (LEFT JOIN)
data = data.merge(pais_provincia, on='pais_provincia', how='left')

Juntando **provincia y ciudad** obtenemos datos estadísticos sobre cada **ciudad**

In [11]:
"CELDA N°11"
#Leemos la tabla externa provincia_ciudad con promedio, medianas y percentiles según ciudad de Argentina y Colombia
provincia_ciudad=pd.read_csv("https://raw.githubusercontent.com/HackSpacePeru/Datasets_intro_Data_Science/master/provincia_ciudad.csv",sep=";")
provincia_ciudad.columns

Index(['provincia_ciudad', 'area_ciudad', 'altura_ciudad', 'habitantes_ciudad',
       'densidad_ciudad', 'promedio_ciudad', 'mediana_ciudad',
       'percentil10_ciudad', 'percentil25_ciudad', 'percentil75_ciudad',
       'percentil90_ciudad'],
      dtype='object')

Para nuestro caso de **predicción de precios** nos enfocaremos **exclusivamente** en las variables que sean relevantes para este objetivo.

Por ello vamos a **filtrar** esas columnas del dataset provincia_ciudad.

In [12]:
"CELDA N°12"
#Filtramos solo las columnas relativas al precio con el método loc
provincia_ciudad = provincia_ciudad.loc[:,['provincia_ciudad',
                                           'promedio_ciudad',
                                           'mediana_ciudad',
                                           'percentil10_ciudad',
                                           'percentil25_ciudad',
                                           'percentil75_ciudad',
                                           'percentil90_ciudad']]

Añadimos **nuevas columnas** al final con ayuda del método **merge**. Previamente **concatenamos la provincia y ciudad**

In [13]:
"CELDA N°13"
#Para concatenar ambas columnas simplemente usamos el operador + con el guión bajo entre comillas
data['provincia_ciudad'] = data['provincia_departamento']+ "_" + data['ciudad']

#Para adjuntar los datos de la tabla externa usamos el método merge especificando left (LEFT JOIN)
data = data.merge(provincia_ciudad, on='provincia_ciudad', how='left')

## IV. Supuestos para Modelos de Regresión

### *Estandarización de las variables numéricas predictoras*

Vamos a **estandarizar** usando StandardScaler modificando la distribución de los datos para asegurar la **normalidad** de los datos.

In [14]:
"CELDA N°14"
#Seleccionamos solo las columnas numéricas de la data total
data_to_standar = data.select_dtypes(include = ["number"])

#Aplicamos la librería StandardScaler
from sklearn.preprocessing import StandardScaler
data_standar = StandardScaler().fit_transform(data_to_standar)
data = pd.DataFrame(data=data_standar, columns=data_to_standar.columns)

### *Multicolinealidad*

Vamos a verificar la **multicolinearidad** utilizando el método del **Variance Inflation Factor(VIF)**.

Para mayor información consultar la siguiente [página](https://towardsdatascience.com/multi-collinearity-in-regression-fe7a2c1467ea)

In [15]:
"CELDA N°15"
#Aplicamos variance_inflation_factor sobre todas las columnas para obtener un dataframe
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["features"] = data.columns
vif["vif_Factor"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
vif

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,features,vif_Factor
0,rooms,4.824583
1,bedrooms,6.871245
2,bathrooms,3.298516
3,surface_total,7.867226
4,price,4.845436
5,promedio_provincia,304.722250
6,mediana_provincia,231.320329
7,percentil10_provincia,114.810387
8,percentil25_provincia,199.430680
9,percentil75_provincia,336.628552


Se evidencia que los datos externos superan por mucho el límite permisible. Así como también algunas variables originales.

Para corregirlo haremos uso del **PCA** (sin considerar el precio).

In [16]:
"CELDA N°16"
#Separamos las variables predictoras del target
X = data.drop('price',axis=1)
y = data['price']

In [17]:
"CELDA N°17"
#Aplicamos PCA sobre las variables predictoras y actualizamos X para obtener un nuevo dataframe de VIF
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
components=pca.fit_transform(X)
X=pd.DataFrame(data=components,columns=['PCA1','PCA2','PCA3','PCA4','PCA5','PCA6'])

vif = pd.DataFrame()
vif["features"] = X.columns
vif["vif_value"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

,features,vif_value
0,PCA1,1.0
1,PCA2,1.0
2,PCA3,1.0
3,PCA4,1.0
4,PCA5,1.0
5,PCA6,1.0


Ahora ya hemos cumplido los **supuestos** para ejecutar un modelo de **regresión**:

* Distribución normal (sin outliers) de las variables predictoras
* Independencia entre las variables predictoras

##  V. Preparamos los datos para el modelo

Finalmente, con ayuda de la librería **train_test_split** dividimos la data de **train**: 85% para entrenamiento y **15%** para validación

In [18]:
"CELDA N°18"
#Con la librería train_test_split generamos los datos de entrenamiento y validación 
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid= train_test_split(X,y,test_size = 0.15,random_state=1) 

# **Modelamiento y Evaluación**

Importamos las librerías de **Scikit Learn** para realizar:
1.   Regresión Lasso
2.   Regresión Ridge


In [19]:
"CELDA N°19"
#Obtenemos los modelos de la librería linear_models de sklearn
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

#Desactivamos los mensajes de advertencia para mejor visibilidad de los resultados de cada celda
import warnings
warnings.filterwarnings("ignore")

In [20]:
"CELDA N°20"
#Utilizaremos  el indicador denominado MSLE para obtener el error logarítmico promedio
from sklearn.metrics import mean_squared_log_error

In [21]:
"CELDA N°21"
#Obtenemos el MSLE aplicando Lasso variando el valor de alpha
for i in range(1,10):
    lasso = Lasso(alpha=i/100).fit(X_train,y_train)
    lasso_predictions=lasso.predict(X_valid)
    print("Mi MSLE es: ", mean_squared_log_error(abs(y_valid),abs(lasso_predictions)), "cuando alpha es: ", i/100)

Mi MSLE es:  0.05450044535546668 cuando alpha es:  0.01
Mi MSLE es:  0.054883735821263074 cuando alpha es:  0.02
Mi MSLE es:  0.05567818466692163 cuando alpha es:  0.03
Mi MSLE es:  0.056846166848497866 cuando alpha es:  0.04
Mi MSLE es:  0.05842181813836395 cuando alpha es:  0.05
Mi MSLE es:  0.06036424753898551 cuando alpha es:  0.06
Mi MSLE es:  0.06271178101536869 cuando alpha es:  0.07
Mi MSLE es:  0.06545731883217724 cuando alpha es:  0.08
Mi MSLE es:  0.06860346677978614 cuando alpha es:  0.09


Para mayor detalle del porqué un valor acordado para **alpha = 0.01** sin basarnos en una base **teórica** acudir al [video](https://youtu.be/4b4MUYve_U8) **minuto 26** de la clase dictada por **AndreNg en Standford**.

In [22]:
"CELDA N°22"
#Obtenemos el MSLE aplicando Lasso variando el valor de alpha
for i in range(1,10):
    ridge = Ridge(alpha=i/100).fit(X_train,y_train)
    ridge_predictions=ridge.predict(X_valid)
    print("Mi MSLE es: ", mean_squared_log_error(abs(y_valid),abs(ridge_predictions)), "cuando alpha es: ", i/100)

Mi MSLE es:  0.054490346410072596 cuando alpha es:  0.01
Mi MSLE es:  0.05449033719651285 cuando alpha es:  0.02
Mi MSLE es:  0.05449032798380175 cuando alpha es:  0.03
Mi MSLE es:  0.05449031877193933 cuando alpha es:  0.04
Mi MSLE es:  0.054490309560925536 cuando alpha es:  0.05
Mi MSLE es:  0.0544903003507604 cuando alpha es:  0.06
Mi MSLE es:  0.054490291141443874 cuando alpha es:  0.07
Mi MSLE es:  0.054490281932975955 cuando alpha es:  0.08
Mi MSLE es:  0.054490272725356656 cuando alpha es:  0.09


# **Cross Validation**

Para evaluar la **robustez** del modelo vamos a utilizar **Repeated KFoldes** para aumentar el número de iteraciones en la divisiión de **KFolds**.

Puedes encontrar mayor información en este [enlace](https://machinelearningmastery.com/repeated-k-fold-cross-validation-with-python/)

In [23]:
"CELDA N°23"
#Aplicamos las librerías correspondientes para ejecutar el Repeated KFoldes y Cross Validation

from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

#Realizamos 5 particiones y 3 repeticiones
cv = RepeatedKFold(n_splits=5, n_repeats=3)

#Guardamos los resultados de aplicar el MSLE en la variable scores
scores = -cross_val_score(Ridge(alpha=0.01), abs(X), abs(y), scoring='neg_mean_squared_log_error', cv=cv, n_jobs=-1)

#Finalmente obtenemos el promedio y desviación estándar de los resultados
print('Promedio y Desviación del Error: %.3f (%.3f)' % (mean(scores), std(scores)))

Promedio y Desviación del Error: 0.083 (0.002)


Lo **ideal** es que el error logarítmico promedio sea **mínimo**, por ello el Promedio debe ser **cercano a cero** (al igual que la desviación).

De no cumplise alguna de estas condiciones es una alerta para mejorar el modelo para que sea más **robusto** y mejore su precisión para diferentes datos.